# Imports

In [1]:
import numpy as np
import pyquasar as pq

# Helmholtz equation with fixed frequency

$\Delta p + k^2 p = -j \rho_0 \omega q$

Dirichlet boundary condition:

$p = \hat p$

Neumann boundary condition:

$\nu_n = \frac{j}{\rho_0 \omega} \frac{\partial p}{\partial n} = \hat \nu_n$

Robin boundary condition:

$p = Z \nu_n = \frac{j}{\rho_0 \omega A} \frac{\partial p}{\partial n}$ 

where:
 - $\omega = 2 \pi f$;
 - $k = \frac{\omega}{c}$;
 - $c$ -- speed of sound;
 - $f$ -- frequency;
 - $\rho_0$ -- density;
 - $Z$ -- impedance;
 - $A$ -- ???;
 - $\nu_n$ -- normal velocity.

## Robin boundary condition

$a u + b \frac{\partial u}{\partial n} = g$

$\frac{\partial u}{\partial n} + \frac{a}{b} u = \frac{g}{b}$

$\lambda \frac{\partial u}{\partial n} + \beta (u - u_{\beta}) = 0$

$\lambda \frac{\partial u}{\partial n} + \beta u = \beta u_{\beta}$


$p = \frac{j}{\rho_0 \omega A} \frac{\partial p}{\partial n}$

$\frac{\partial p}{\partial n} - \frac{\rho_0 \omega A}{j} p = 0$

In [15]:
air = {"density": 1.21, "speed": 343, "impedance": 415} 

In [57]:
f = 10000
k = 2 * np.pi * f / air["speed"]
j = 1.j

In [58]:
def u(p, n):
  return p[..., 0] - p[..., 1]

def rhs(p, n):
  return k**2 * (p[..., 0] - p[..., 1])

def flow(p, n):
  return n[..., 0] - n[..., 1]

def ub(p, n):
  return 2 * (n[..., 0] - n[..., 1]) + p[..., 0] - p[..., 1]

In [59]:
mesh = pq.Mesh.load("cube.geo", refine_k=1)
domains = [pq.FemDomain(domain) for domain in mesh]
problem = pq.FemProblem(domains)

In [60]:
mesh

<Mesh object summary 
	Numeration: global
	Domains: [<MeshDomain object summary
	Material: air
	Total elements number: 3776
	Element type: Tetrahedron 4; Count: 3776
	Boundary type: dirichlet; Tag: 1; Element type: Triangle 3; Count: 200.
	Boundary type: dirichlet; Tag: 2; Element type: Triangle 3; Count: 136.
	Boundary type: dirichlet; Tag: 3; Element type: Triangle 3; Count: 200.
	Boundary type: dirichlet; Tag: 4; Element type: Triangle 3; Count: 136.
	Boundary type: robin; Tag: 5; Element type: Triangle 3; Count: 256.
	Boundary type: robin; Tag: 6; Element type: Triangle 3; Count: 256.
>]>

In [61]:
materials = {"air": {"air": rhs, "robin": {"robin":ub, "beta": 0.5}, "gamma": k**2}}
problem.assembly(materials, dtype=np.complex128)

In [62]:
problem.add_skeleton_projection(u, ["dirichlet"], dtype=np.complex128)

In [63]:
sol = problem.solve(1e-15)

In [64]:
real_sol = sol.real

In [65]:
np.linalg.norm(sol - u(domains[0].vertices, 0)) / np.linalg.norm(u(domains[0].vertices, 0))

1.5831658352453858e-12

In [66]:
np.linalg.norm(real_sol - u(domains[0].vertices, 0)) / np.linalg.norm(u(domains[0].vertices, 0))

1.5831658352453854e-12

In [67]:
mic_pos = np.array([[2, 2, 2]])

In [68]:
proj = problem.tabulate(mic_pos, batch_size=1)

100%|██████████| 1/1 [00:00<00:00, 277.09it/s]


In [69]:
proj @ real_sol

array([1.17705325e-14])